# **Kalimantan Solar Power Analysis**

## NASA Power Data Crawling

In [ ]:
import json
import urllib3
import time

#ini fungsi bikin url
def url (Lat, Lon):
    
    SD = 20140901
    ED = 20190831
    NewBaseUrl = 'https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?'

    Request = 'request=execute'

    Identifiers = 'identifier=SinglePoint'
    #Regional untuk area, SinglePoint untuk titik

    Parameters = 'parameters=T2M,ALLSKY_SFC_SW_DWN,QV2M,RH2M'
    # list of parameter available here: https://power.larc.nasa.gov/docs/v1/#examples
    
    Duration='startDate='+str(SD)+'&'+'endDate='+ str (ED)
    #replace the YYYYMMDD with the duration

    UserCommunity = 'userCommunity=SSE'
    #SSE (Surface meteorology and Solar Energy),SB(sustainable Buildings), AG(Agroclimatology))

    TemporalAverage='tempAverage=DAILY'

    Output = 'outputList=JSON'
    #CSV,ASCII, JSON, etc

    Coordinates = 'lat='+str(Lat)+'&'+'lon='+ str(Lon)
    #La = Latitude, Lon = Longitude

    User = 'user=anonymous'

    Url = NewBaseUrl+Request+'&'+Identifiers+'&'+Parameters+'&'+Duration+'&'+UserCommunity+'&'+TemporalAverage+'&'+Output+'&'+Coordinates+'&'+User
    time.sleep(30)
    print (Url)
    
    http = urllib3.PoolManager()
    read = http.request ('GET', Url)
    Namafile = str (Lat) + ','+ str (Lon)
    #DataNasa = pd.read_json(Url)
    DataNasa = json.loads(read.data.decode('utf-8'))
    #read.to_json(Namafile+'.json')
    with open (Namafile+'.json','w') as myfile:
        json.dump(DataNasa,myfile)

#Generate for all Coordinate in Kalimantan
# Fungsi Box 1, nlat = total pengulangan latitude (22), nlon total pengulangan longitude (40)
def box_1 (nlat,nlon):
    int (nlat)
    int (nlon)
    Lat = -1.5989
    #0.7659
    Latakhir = -2.4852
    inclat = (Latakhir-0.7659)/nlat
    rinclat = round (inclat,4)
    Lon = 112.3385
    #110.4106
    #110.2623
    Lonakhir = 116.1950
    inclon = (Lonakhir-110.2623)/nlon
    rinclon = round(inclon,4)
    ilat = 1
    ilon = 1
    i = 0
    while ilat <= nlat :
        while ilon <= nlon:
            Lon = Lon + rinclon
            Lon = round (Lon,4)
            ilon = ilon+1
            url(Lat,Lon)
            #print (str (Lat)+','+str(Lon))
            i = i+1
            print (i)
        ilat = ilat+1
        ilon = 1
        Lat = Lat+rinclat
        Lat = round (Lat,4)
        Lon = 112.3385
        #110.4106
        #110.2623
    return;

# Fungsi Box 2, nlat = total pengulangan latitude (40), nlon total pengulangan longitude (10)
def box_2 (nlat,nlon):
    int (nlat)
    int (nlon)
    #4.2995
    Lat = 2.39
    Latakhir = -0.7941
    inclat = (Latakhir-4.2995)/nlat
    rinclat = round (inclat,4)
    Lon = 116.7114
    #116.2389
    Lonakhir = 117.1837
    inclon = (Lonakhir-116.2389)/nlon
    rinclon = round(inclon,4)
    ilat = 1
    ilon = 1
    i = 0
    while ilat <= nlat :
        while ilon <= nlon:
            Lon = Lon + rinclon
            Lon = round (Lon,4)
            ilon = ilon+1
            url(Lat,Lon)
            #print (str (Lat)+','+str(Lon))
            i = i+1
            print (i)
        ilat = ilat+1
        ilon = 1
        Lat = Lat+rinclat
        Lat = round (Lat,4)
        Lon = 116.7114
        #116.2389
    
    return;

# Fungsi Box 2, nlat = total pengulangan latitude (10), nlon total pengulangan longitude (10)
def box_3 (nlat,nlon):
    int (nlat)
    int (nlon)
    #0.8978
    #-0.0402
    Lat = -0.0402
    Latakhir = -0.4425
    inclat = (Latakhir-0.8978)/nlat
    rinclat = round (inclat,4)
    Lon = 109.744
    #109.744
    #109.2516
    Lonakhir = 110.4821
    inclon = (Lonakhir-109.2516)/nlon
    rinclon = round(inclon,4)
    ilat = 1
    ilon = 1
    i = 0
    while ilat <= nlat :
        while ilon <= nlon:
            Lon = Lon + rinclon
            Lon = round (Lon,4)
            ilon = ilon+1
            #url(Lat,Lon)
            #print (str (Lat)+','+str(Lon))
            i = i+1
            print (i)
        ilat = ilat+1
        ilon = 1
        Lat = Lat+rinclat
        Lat = round (Lat,4)
        Lon = 109.744
        #109.2516
    return;
    
box_1(22,40)
box_2 (40,10)
box_3(10,10)


## Creating Dataframe

In [ ]:
import os
import json
import csv
import pandas as pd

def outputdaya(Hum,Temp,ASky):
    #rumus menghitung effiesiensi pv panel efek temp dan humidity
    eff = (((Hum*-0.1518)+17.222-((Temp-25)*0.5))/100)
    #print (eff)
    output = ASky*eff/24*1000
    return (output)
    #print(output)
def effisiensi (Hum,Temp):
    eff = (((Hum*-0.1518)+17.222-((Temp-25)*0.5))/100)
    return (eff)

IsiFile = os.scandir('D:/Data Project DTS/')
ListFile = []
ListLatitude = []
ListLongitude = []
ListT2M = []
ListAllSky = []
ListQV2M = []
ListRH2M = []
ListOutput = []
ListEfisiensi = []
ListCoordinate = []
IndexData = 0
FileKe = 1

with IsiFile as Entries:
    for entry in Entries:
        if entry.is_file():
            ListFile.append(entry.name)
for Namafile in ListFile:
    with open('D:/Data Project DTS/'+Namafile) as f:
        print ('Nama File ', Namafile, ' File ke ', FileKe)
        data = json.load(f)
        features = data['features']
        ValueFeatures = features[0]
        Geometry = ValueFeatures ['geometry']
        Coordinate = Geometry ['coordinates']
        #Format Coordinate dalam bentuk list [lon, Lat, Belum tau ini apa, altitude mungkin]
        CoordinateLat = Coordinate[1]
        ListLatitude.append(CoordinateLat)
        CoordinateLon = Coordinate[0]
        ListLongitude.append(CoordinateLon)
        Coordinate = str (CoordinateLat)+ ',' + str (CoordinateLon)
        ListCoordinate.append (Coordinate)
        Properties = ValueFeatures['properties']
        Parameter = Properties['parameter']
        ParameterKey = ['T2M','ALLSKY_SFC_SW_DWN','QV2M','RH2M']
        for Key in ParameterKey:
            DateData= Parameter[Key]
            ListValue = list (DateData.values())
            for Value in ListValue:
                if Value == -999:
                    ListValue.remove(-999)
                    continue
            SumofData = 0
            for Value in ListValue:
                SumofData = SumofData+Value
            AverageofData = SumofData/len(ListValue)
            if Key == 'T2M':
                AverageofData4 = round (AverageofData,4)
                ListT2M.append(AverageofData4)
            elif Key == 'ALLSKY_SFC_SW_DWN' :
                AverageofData4 = round (AverageofData,4)
                ListAllSky.append(AverageofData4)
            elif Key == 'QV2M':
                AverageofData4 = round (AverageofData,4)
                ListQV2M.append(AverageofData4)
            elif Key == 'RH2M':
                AverageofData4 = round (AverageofData,4)
                ListRH2M.append(AverageofData4)
            else:
                continue
        print ("Assign Data Complete")
        FileKe= FileKe+1
        
print ('All File Complete')
while IndexData < len (ListFile):
    print ('Ini index data ke ', IndexData)
    DayaPV = round (outputdaya(ListRH2M[IndexData],ListT2M[IndexData],ListAllSky[IndexData]),4)
    EffisiensiPV = round (effisiensi (ListRH2M[IndexData],ListT2M[IndexData]),4)
    ListOutput.append(DayaPV)
    ListEfisiensi.append(EffisiensiPV)
    IndexData = IndexData+1
print ("Creating Data Frame")    
MeanDataNasa = {'Latitude' : ListLatitude, 'Longitude' : ListLongitude, 'Coordinate': ListCoordinate, 'Temperature at 2 Meters' : ListT2M,'All Sky Insolation Incident on a Horizontal Surface' : ListAllSky, 'Specific Humidity at 2 Meters' : ListQV2M, 'Relative Humidity at 2 Meters' : ListRH2M, 'Efisiensi' : ListEfisiensi, 'Output' : ListOutput }
#print (df)
df = pd.DataFrame(MeanDataNasa)
print ("Data Frame Created")
df = df.sort_values(by='Output', ascending=False)
print ("Sorting berhasil")
df.to_csv('D:/Data Project DTS/Final dengan efisiensi.csv', index = False)
print ("File Created")


## Visualizing Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

dataku=pd.read_csv("dataku.csv", sep=';')
dataku.head()

Temp = dataku['Temp']
RH = dataku['RH']
Eff = dataku['Eff']
Output = dataku['Output']
SA = dataku['SA']

angka = np.arange(0,1399)

plt.figure()
plt.scatter(angka, Temp)
plt.title("Sebaran nilai Temperature")
plt.ylabel("Temperature")
plt.show()

plt.figure()
plt.scatter(angka, RH)
plt.title("Sebaran nilai Kelembaban Relatif")
plt.ylabel("Kelembaban Relatif")
plt.show()

plt.figure()
plt.scatter(angka, SA)
plt.title("Sebaran nilai Radiasi Matahari")
plt.ylabel("Radiasi Matahari")
plt.show()

dataku.sort_values(by='Temp', ascending=True)
plt.figure()
plt.scatter(Temp, Eff)
plt.xlabel("Temperature")
plt.ylabel("Effisiensi Solar Panel")
plt.show()

dataku.sort_values(by='RH', ascending=True)

plt.figure()
plt.scatter(RH, Eff)
plt.xlabel("Kelembaban Relatif")
plt.ylabel("Effisiensi Solar Panel")
plt.show()

## Final Visualization

We created the final visualization with an interactive dashboard at http://tiny.cc/AstroGroupBDA1 